In [1]:
from pyspark.sql import functions as F

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1681596999367_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from py4j.protocol import Py4JJavaError
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import explode, col, udf, concat_ws, from_json, lit, array, expr, size
from pyspark.sql.functions import sum as _sum
from pyspark.sql.types import *
import json
import os
from pyspark.sql.types import BooleanType
import sys
import gc
print(sys.version)
# conf = SparkConf()
# sc = SparkContext(conf=conf)
# sc.setLogLevel("ERROR")

# spark = SparkSession.builder.appName("twitter_applications")\
#     .config("spark.sql.files.ignoreCorruptFiles", "true")\
#     .config('spark.executor.memory', '45G')\
#     .config('spark.driver.memory', '80G')\
#     .config('spark.driver.maxResultSize', '10G')\
#     .getOrCreate()

'''
    Step 3:
    Implement basic PageRank as described in 
    http://www.diva-portal.org/smash/get/diva2:1104337/FULLTEXT01.pdf Section 3.4.1
'''
input_bucket = 's3://cs5344-twitter-project/input/'
user_pair_rdd = spark.read \
     .csv(input_bucket+"user_pair_aws/*.csv").rdd
user_pair_rdd = user_pair_rdd.map(lambda x: (x[0], (x[1], x[2])))
user_interact_rdd = spark.read \
     .csv(input_bucket+"user_interact_aws/*.csv").rdd
id_name = spark.read.option("header",True).csv(input_bucket+"id_name_dict_aws/*.csv")
id_name.show()
# if user A retweet B, the from_user = A, to_user = B
columns = ['from_user', 'to_user', 'retweet_ratio']
edge_rdd = user_pair_rdd.join(user_interact_rdd).map(lambda x: (x[0], x[1][0][0], float(x[1][0][1]) / float(x[1][1]))) \
    .filter(lambda x: x[0] != x[1]).toDF(columns)  # remove all self pointed edges

all_users = edge_rdd.select("from_user").union(edge_rdd.select("to_user")).distinct() \
    .withColumnRenamed("from_user", "users")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3.7.16 (default, Dec 15 2022, 23:24:54) 
[GCC 7.3.1 20180712 (Red Hat 7.3.1-15)]
+-------------------+-------------------+
|        user_id_str|user_twitter_handle|
+-------------------+-------------------+
|1277653717467172864|          KonateEra|
| 889184194315866113|     ViniFerreirado|
|          296488613|              tenoq|
|1292411311612030977|    kamikaze1226617|
|          224175846|           Edunwa18|
|1181834654317531136|         MKtography|
|1241239744316690432|    SpeedDe86282699|
|          201088131|         Fiberblock|
|         4149402979|        luislara_97|
|         2418331615|            eptileR|
|1484471406985297925|           WabinabG|
|1278814483167633409|          jhendryx3|
| 934943098374823936|          shige_e_5|
|1475408590445989894|         TAzammalik|
|         2613144614|         AUTOMYSTIC|
|1482121649650548736|        alexhaujshu|
|1473274640805994497|    hoki2022bigwinx|
|         3173560541|      AshfordWomens|
|1233591105410625536|          jmfodd

In [3]:
# count of all nodes in the graph, use to init the pagerank score
N = all_users.count()
print("number of users in the graph: " + str(N))
all_users = all_users.withColumn("rank", lit(1 / N))
damping_factor = 0.85

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

number of users in the graph: 838508

In [10]:
# keep update the score until convergence
for cycle in range(10):
    temp = all_users.join(edge_rdd, all_users.users == edge_rdd.to_user, "left").na.fill(value=0,
                                                                                         subset=["retweet_ratio"])
    temp.show()
    temp = temp.withColumn("contri", temp.retweet_ratio * temp.rank)
    all_users = temp.groupby("users").agg(_sum("contri").alias("rank"))#*lit(damping_factor) + lit((1 - damping_factor) / N)
    all_users = all_users.withColumn("rank", all_users.rank *lit(damping_factor) + lit((1 - damping_factor) / N)) #* damping_factor + (1-damping_factor) /N
    norm = all_users.rdd.map(lambda x: (1, x[1])).reduceByKey(lambda a, b: a + b).collect()[0][1]
    all_users = all_users.withColumn("rank", all_users.rank / norm)
    del norm
    gc.collect()
    # all_users.sort(col("rank").desc()).show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------------------+-------------------+-------------------+-------------+
|              users|                rank|          from_user|            to_user|retweet_ratio|
+-------------------+--------------------+-------------------+-------------------+-------------+
|1000011006725115905|1.475353053910136...|               null|               null|          0.0|
|1000017857009209344|1.475353053910136...|               null|               null|          0.0|
|1000057319806390272|1.475353053910136...|               null|               null|          0.0|
|1000067944028745730|1.475353053910136...|               null|               null|          0.0|
|          100017393|1.475353053910136...|               null|               null|          0.0|
|1000209664364220416|1.475353053910136...|               null|               null|          0.0|
|1000328182208303104|1.499320523751682...|          129065022|1000328182208303104|          1.0|
|1000342261744349185|1.4753530

In [11]:
results = all_users.sort(col("rank").desc()).limit(100).join(id_name, id_name.user_id_str == all_users.users, "left") \
    .select("users", "rank", "user_twitter_handle")
results.sort(col("rank").desc()).show(50)

# write to CSV
results.sort(col("rank").desc()).limit(50).write.mode('overwrite').csv("s3://cs5344-twitter-project/test_pokemon_damping_new")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------------------+-------------------+
|              users|                rank|user_twitter_handle|
+-------------------+--------------------+-------------------+
|          130447415|  0.9999865664154867|            hikakin|
|         2786677338|1.539866559174283...|    denfaminicogame|
|          307902310|2.102073488707743...|           Nintendo|
| 811765148595535872|2.362355445346533...|    GAME_KNIVES_OUT|
|          323091128|1.878467721079452...|       Pokemon_cojp|
|           29758446|1.852907679985346...|      RockstarGames|
| 909119962572242944|1.840777642711827...|        sawada46493|
|            5162861|1.834773954515961...|    NintendoAmerica|
| 987228577522376704|1.819243510795801...|         poke_times|
|1470439914953011204|1.806474032953455...|       DarkLandGame|
|         3095001124|1.773756286132017...|         OoCPokemon|
|1349579108187992071|1.772904255355948...|     tensura_m_game|
|1441557548427927554|1.766371267263846...|      Gamersn

In [ ]:
spark.stop()